In [4]:
import numpy as np
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
import pandas as pd



In [7]:
df=pd.read_csv(r"C:\Users\anjan\Desktop\NER_Dataset.csv")

In [10]:
df.head()

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."


In [8]:
sentences = []
tags = []
for i, row in df.iterrows():
    sentence = row['sentence']
    sentence_tokens = [stemmer.stem(word.lower()) for word in word_tokenize(sentence) if word.lower() not in stop_words]
    sentences.append(sentence_tokens)
    tags.append([tag.strip() for tag in row['tags'].split(',')])


KeyError: 'sentence'

In [ ]:
# Build the tag transition probabilities
tag_transition_prob = defaultdict(lambda: defaultdict(float))
for sentence, sentence_tags in zip(sentences, tags):
    for i in range(len(sentence_tags) - 1):
        tag_transition_prob[sentence_tags[i]][sentence_tags[i+1]] += 1

for tag, next_tag_prob in tag_transition_prob.items():
    total = sum(next_tag_prob.values())
    for next_tag, count in next_tag_prob.items():
        tag_transition_prob[tag][next_tag] /= total

# Define a function to find the most probable sequence of tags for a given jumbled sentence
def find_tag_sequence(jumbled_sentence):
    jumbled_tokens = [stemmer.stem(word.lower()) for word in word_tokenize(jumbled_sentence) if word.lower() not in stop_words]
    
    # Initialize the Viterbi table and backpointer table
    viterbi = np.zeros((len(jumbled_tokens), len(set(tag for tags in tags for tag in tags))))
    backpointer = np.zeros((len(jumbled_tokens), len(set(tag for tags in tags for tag in tags))), dtype=int)
    
    # Iterate through the tokens in the jumbled sentence
    for i, token in enumerate(jumbled_tokens):
        # Find all possible tags for the current token
        possible_tags = set()
        for sentence_tags in tags:
            for tag in sentence_tags:
                if token in [stemmer.stem(word.lower()) for word in sentences[i]]:
                    possible_tags.add(tag)
        
        # Initialize the Viterbi and backpointer tables for the first token
        if i == 0:
            for j, tag in enumerate(possible_tags):
                viterbi[i, j] = tag_transition_prob['<start>'][tag]
                backpointer[i, j] = -1
        # Update the Viterbi and backpointer tables for subsequent tokens
        else:
            for j, tag in enumerate(possible_tags):
                max_prob = 0
                max_prev_tag_idx = -1
                for k, prev_tag in enumerate(possible_tags):
                    prob = viterbi[i-1, k] * tag_transition_prob[prev_tag][tag]
                    if prob > max_prob:
                        max_prob = prob
                        max_prev_tag_idx = k
                viterbi[i, j] = max_prob
                backpointer[i, j] = max_prev_tag_idx
    
    # Find the most probable sequence of tags
    most_probable_tags = []
    current_tag_idx = np.argmax(viterbi[-1, :])
    for i in range(len(jumbled_tokens)-1, -1, -1):
        most_probable_tags.insert(0, list(possible_tags)[current_tag_idx])
        current_tag_idx = backpointer[i, current_tag_idx]
    
    return most_probable_tags

# Example usage
jumbled_sentence = "The quick brown fox jumps over the lazy dog."
most_probable_tags = find_tag_sequence(jumbled_sentence)
print(f"Most probable tags for the jumbled sentence: {', '.join(most_probable_tags)}")

In [11]:
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Load the dataset
df = pd.read_csv(r"C:\Users\anjan\Desktop\NER_Dataset.csv")

In [23]:
# Preprocess the dataset
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

sentences = []
pos_tags = []
tags = []
for _, row in df.iterrows():
    sentence = row['Word'].split(',')
    sentence = [stemmer.stem(word.lower()) for word in sentence if word.lower() not in stop_words]
    pos_tag = row['POS'].split(',')
    tag = row['Tag'].split(',')
    sentences.append(sentence)
    pos_tags.append(pos_tag)
    tags.append(tag)
    # Build the tag transition probabilities
tag_transition_prob = defaultdict(lambda: defaultdict(float))
for sentence_tags in tags:
    for i in range(len(sentence_tags) - 1):
        tag_transition_prob[sentence_tags[i]][sentence_tags[i+1]] += 1

for tag, next_tag_prob in tag_transition_prob.items():
    total = sum(next_tag_prob.values())
    for next_tag, count in next_tag_prob.items():
        tag_transition_prob[tag][next_tag] /= total



In [24]:
# Define a function to find the most probable sequence of tags for a given jumbled sentence
def find_tag_sequence(jumbled_sentence):
    jumbled_tokens = [stemmer.stem(word.lower()) for word in word_tokenize(jumbled_sentence) if word.lower() not in stop_words]
    
    # Initialize the Viterbi table and backpointer table
    viterbi = np.zeros((len(jumbled_tokens), len(set(tag for tags in tags for tag in tags))))
    backpointer = np.zeros((len(jumbled_tokens), len(set(tag for tags in tags for tag in tags))), dtype=int)
    
    # Iterate through the tokens in the jumbled sentence
    for i, token in enumerate(jumbled_tokens):
        # Find all possible tags for the current token
        possible_tags = set()
        for sentence, sentence_tags in zip(sentences, tags):
            if token in sentence:
                for j, word in enumerate(sentence):
                    if word == token:
                        possible_tags.add(sentence_tags[j])
        
        # Initialize the Viterbi and backpointer tables for the first token
        if i == 0:
            for j, tag in enumerate(possible_tags):
                viterbi[i, j] = tag_transition_prob['<start>'][tag]
                backpointer[i, j] = -1
        # Update the Viterbi and backpointer tables for subsequent tokens
        else:
            for j, tag in enumerate(possible_tags):
                max_prob = 0
                max_prev_tag_idx = -1
                for k, prev_tag in enumerate(possible_tags):
                    prob = viterbi[i-1, k] * tag_transition_prob[prev_tag][tag]
                    if prob > max_prob:
                        max_prob = prob
                        max_prev_tag_idx = k
                viterbi[i, j] = max_prob
                backpointer[i, j] = max_prev_tag_idx
    
    # Find the most probable sequence of tags
    most_probable_tags = []
    current_tag_idx = np.argmax(viterbi[-1, :])
    for i in range(len(jumbled_tokens)-1, -1, -1):
        most_probable_tags.insert(0, list(possible_tags)[current_tag_idx])
        current_tag_idx = backpointer[i, current_tag_idx]
    
    return most_probable_tags


In [25]:
# Example usage
jumbled_sentence = "The quick brown fox over jumps the lazy dog"
most_probable_tags = find_tag_sequence(jumbled_sentence)
print(f"Most probable tags for the jumbled sentence: {', '.join(most_probable_tags)}")

IndexError: list index out of range